## FEATURE IMPORTANCE & SELECTION

CONTEXT 

with 400+ features, not all contribute meaningfully to predictions. this will be a baseline model to estimate feature importance, enabling informed decision about which features to keep , drop , or engginer further

objective 
- train a quick LightGBM Model for importance estimation
- identify zero importance and low improtance features
- select top N features for downstream modeling
- save the feature importance ranking for reference

In [1]:
#import and load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import lightgbm as lgb

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

data_path = Path('../data/processed/train_features.parquet')
if not data_path.exists():
    data_path = Path('../data/interim/train_merged.parquet')

train = pd.read_parquet(data_path)
print(f'Data loaded: {train.shape}')

Data loaded: (590540, 434)


In [5]:
# prepare features

exclude_cols = ['TransactionID', 'isFraud', 'TransactionDT']
object_cols = train.select_dtypes(include=['object']).columns.to_list()

feature_cols = [c for c in train.columns if c not in exclude_cols and c not in object_cols]

X = train[feature_cols]
y = train['isFraud']

print(f"feature : {len(feature_cols)}")
print(f'target : {y.sum():,} fraud / {len(y):,} total ({y.mean()*100:.2f}%)')

feature : 431
target : 20,663 fraud / 590,540 total (3.50%)


In [ ]:
# lightGBM for importance

params = {
    'objective' : 'binary',
    'metric' : 'auc',
    'boosting_type' : 'gdbt',
    'num_leaves' : 31,
    'learning_rate' : 0.1,
    'feature_fraction' : 0.8,
    'bagging_fraction' : 0.8,
    'bagging_freq' : 
    }